In [157]:
#TO DO
#Save a current version of HTML each time scrape so can do a div comparison whenever this script breaks to see how their html has changed
# Might be able to replace the sleep for 5 seconds part of the scrape function with webdriver wait, which wsits for javascript to load so dont have to have extra buffer time on each scrape

In [158]:
#run thredup_fullscrape.py

In [178]:
!/usr/bin/env python3
!pip install tqdm

Python 3.8.12 | packaged by conda-forge | (default, Jan 30 2022, 23:13:55) 
[Clang 11.1.0 ] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> 
KeyboardInterrupt
>>> 
>>> Collecting tqdm
     |████████████████████████████████| 78 kB 6.2 MB/s             


In [179]:
#import requests
import time
import random
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from alive_progress import alive_bar
import re
import requests # to get image from the web
import shutil # to save it locally
from os.path import exists
from os.path import expanduser
from tqdm import tqdm
import json


# CURRENT CODE

#NOTE TO SELF: If we want 10,000 items from four categories (shirts, sweaters, dresses, outwerwear), we need about 2,500 per category

so set max_products = 2,500
max_pages = 21 (bc 120 items per pages as default)

* For each front_page_url, make sure to save a copy of the link used here and it should have items ordered from newest to oldest

In [205]:
save_folder = "/Users/karina/Desktop/FashionSave/"
max_products = 3 #set to "all" if want to take all products on scraped pages
front_page_url = "https://www.thredup.com/women/outerwear?category_tags=outerwear&department_tags=women&sort=newest_first&page=1"
max_pages = 10

In [185]:
#SCRAPE PAGE FUNCTION

def scrape_page(my_url):
    print("scraping url")
    firefox_options = webdriver.FirefoxOptions()
    #firefox_options.add_argument('--headless')
    firefox_options.add_argument('window-size=1920x1080');
    driver = webdriver.Firefox(options=firefox_options)
    driver.get(my_url)
    time.sleep(10)
    page_contents=driver.page_source
    page_contents = BeautifulSoup(page_contents, 'html.parser')
    driver.quit()
    return page_contents
    

In [186]:
#GET LINKS TO INDIVIDUAL ITEM PAGES FROM MAIN PAGE

def get_item_links(front_page_url, max_pages = 1):
    product_links = []
    # Everytime range increases, items increase by 50.
    for page_number in range(1, max_pages + 1):
        print(f"KSN: Page {page_number}...")
        #See if page exists
        try:
            url_page = front_page_url + "&page=" + str(page_number)
            main_page_items = scrape_page(url_page)
        except:
            print('exception')
            break #exit for loop if a page doesn't exist (presumably means past last page for this item) 

        #Pull all href links
        url_front = "https://www.thredup.com"
        hrefs = []
        product_list = []
        all_products = main_page_items.find_all(attrs={"data-inp-label": "link-item-card"})


        for product in all_products: #get all product links
            product_link = url_front + product["href"]
            product_links.append(product_link)
            
        return product_links

            
product_links = get_item_links(front_page_url, max_pages)


print(f"Product links found")


KSN: Page 1...
scraping url
Product links found


In [187]:
#Get Product Id from product page (prepare function)

def get_product_id(product_link):
    product_id = product_link.split("?query")[0]   
    product_id = product_id.split("/")[-1]
    
    return product_id

    

In [188]:
#Get product image from product page (prepare functions)

def get_image_link(scraped_page):

    imageLinks = []
    images = scraped_page.findAll(
        "img", {"class": lambda L: L and L.startswith("u-rounded-4 u-cursor-pointer")} 
    )
        
    image_link = images[0].get("src")
    print(image_link)
    return image_link

def save_image(image_link, product_id, save_folder):
    filename = save_folder + "item" + product_id +".jpg"

    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_link, stream = True)

     # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True
        # Open a local file with wb ( write binary ) permission.
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)
        print("image saved")
    else:
        print('Image Couldn\'t be retrieved')

In [189]:
def get_size_and_brand(scraped_page):
    size = scraped_page.find_all("div", attrs={"class": "u-text-16"}) #structure of find_all input is (element, attrs={})
    brand = size[0].parent.find_previous_sibling().find("a")["title"]
    size = size[0].text
    print(size)
    print(brand)
    return size, brand


In [190]:
def get_category(scraped_page):
    category = scraped_page.find_all("span", attrs={"class": "u-text-16"}) #structure of find_all input is (element, attrs={})
    category = category[0].text
    print(category)
    return category

In [191]:
def get_price(scraped_page):
    price = scraped_page.find_all("span", attrs={"class": "price"}) 
    price = price[0].text
    print(price)
    return price

In [192]:
def get_condition(scraped_page):
    condition_header = scraped_page.find("h2", string="Condition")
    condition = condition_header.find_next_sibling()
    condition = condition.text
    print(condition)
    return condition

In [193]:
def get_material_and_features(scraped_page):
    details_header = scraped_page.find("h2", string="Item details")
    details = details_header.parent.find_next_sibling().find_all('li')
    material = details[0].text
    features = details[1].text
    
    print(material)
    print(features)

    return material, features


In [208]:
def get_measurements_and_fit(scraped_page):
    try:
        measurements_and_fit_header = scraped_page.find("h2", string="Size & fit")
        measurements_and_fit = measurements_and_fit_header.find_next_sibling().find_all('li')
        measurements = measurements_and_fit[0].text
        measurements = measurements.replace("How we measure","")
        fit = measurements_and_fit[1].text
    except:
        measurements = None
        fit = None
    
    print(measurements)
    print(fit)

    return measurements, fit

In [209]:
#Get all product info from product page

def get_product_info(product_link, save_folder):
    
    scraped_page = scrape_page(product_link)

    product_id = get_product_id(product_link)
       
    #Get and Save Image
    image_link = get_image_link(scraped_page)
    save_image(image_link, product_id, save_folder)
    
    #Get Size
    size, brand = get_size_and_brand(scraped_page)
    
    #Get item type
    category = get_category(scraped_page)
    
    #Get Condition
    condition = get_condition(scraped_page)
    
    #Get Material and Features
    material, features = get_material_and_features(scraped_page)
    
    #Get Size and Fit
    measurements, fit = get_measurements_and_fit(scraped_page)
    
    
    product_dict = {
        "product_link" : product_link,
        "product_id" : product_id,
        "size" : size,
        "brand" : brand,
        "category" : category,
        "condition" : condition,
        "material" : material,
        "features" : features,
        "measurements" : measurements,
        "fit" : fit
        
    }
    
    
    return product_dict


In [211]:
#PRODUCT LOOP TO GET ALL OF EACH PRODUCT'S INFO

if max_products == "all":
    max_products = len(product_links)

products = []
for product_link in tqdm(product_links[:max_products]):
    print(f"--------------\ngetting info for product: {product_link}")
    product = get_product_info(product_link, save_folder)
    products.append(product)
    #Pause for random duration to not trigger bot blocker
    time.sleep(random.randrange(5, 10))
    


getting product info
scraping url
https://cf-assets-thredup.thredup.com/assets/556351003/xlarge.jpg
image saved
Size 4
Kay Unger
Jacket
excellent
56% cotton, 22% polyamide, 20% viscose, 2% polyester
3/4 sleeve, jacket, marled, frayed edges, fringe & frayed accents, multi color, white, cropped top
None
None
getting product info
scraping url
https://cf-assets-thredup.thredup.com/assets/549052378/xlarge.jpg
image saved
Size XXS
Vince.
Jacket
very good
36% merino wool, 26% nylon, 18% alpaca, 18% viscose, 2% spandex
Long sleeve, coat, solid, black, knee length
None
None
getting product info
scraping url
https://cf-assets-thredup.thredup.com/assets/502740580/xlarge.jpg
image saved
Size M
Athleta
Track Jacket
very good
100% polyester
Solid, zipper detail, metal accents, black, classic
None
None


In [220]:
#Save json

# Specify the filename
filename = 'scraped_product_details.json'

with open(save_folder + filename, 'w') as f:
    json.dump(products, f, indent=4)
